In [1]:
from fatsecret import Fatsecret
from credentials import customerKey, customerSecret, awsKey, awsSecret
from datetime import timedelta
import datetime
from dateutil.relativedelta import relativedelta
import json

In [6]:
def returnDates(start_date):
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    
    yesterday = datetime.datetime.now() - timedelta(days=3)
    dates_list = []
    current_date = start_date
    while current_date <= yesterday:
        dates_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)
    
    return dates_list

In [7]:
days = returnDates("2024-01-01")

In [4]:
foodEntryAll = []

In [27]:
len(foodEntryAll)

770

In [25]:
fs = Fatsecret(customerKey, customerSecret)
print(fs.get_authorize_url())

AccessCode = input("Enter the access code: ")

fs.authenticate(AccessCode)

https://www.fatsecret.com/oauth/authorize?oauth_token=7e9f782b09e64339ad71512075607e65
Enter the access code: 1816212


('828e31def45e4bf29cd43d3d88c5b5d7', '3b00bbf54f57467ab45843ad8443c014')

In [26]:
for day in days[60:]:
    date = datetime.datetime.strptime(day, '%Y-%m-%d')
    foodEntryDay = fs.food_entries_get(date= date)
    foodEntryAll.extend(foodEntryDay)
fs.close()

In [29]:
with open ("local_warehouse/foodEntries.json", "w") as file:
    json.dump(foodEntryAll, file, indent = 4)

In [30]:
months = ["2024-01-01", "2024-02-01", "2024-03-01"]

In [31]:
weights = []
for month in months:
    weightMonth = fs.weights_get_month(date = datetime.datetime.strptime(month, '%Y-%m-%d'))
    weights.extend(weightMonth)

In [36]:
profile = fs.profile_get()
profile["profile_id"] = 1

In [37]:
for weight in weights:
    weight["BMR"] = 88.362 + (13.397 * float(weight["weight_kg"])) + (4.799 * float(profile["height_cm"])) - (5.677 * 23)

In [38]:
with open ("local_warehouse/weights.json", "w") as file:
    json.dump(weights, file, indent = 4)

In [39]:
with open ("local_warehouse/profile.json", "w") as file:
    json.dump(profile, file, indent = 4)

In [40]:
caloriesBurned = []

In [41]:
for month in months:
    caloriesMonth = fs.exercise_entries_get_month(datetime.datetime.strptime(month, '%Y-%m-%d'))
    if type(caloriesMonth) == dict:
        caloriesBurned.append(caloriesMonth)
    elif type(caloriesMonth) == list:
        caloriesBurned.extend(caloriesMonth)

In [42]:
with open ("local_warehouse/caloriesBurned.json", "w") as file:
    json.dump(caloriesBurned, file, indent = 4)